In [17]:
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import pandas as pd
import time

In [18]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

# NASA Mars News
Scrape the NASA Mars News Site (https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [83]:
mars_stuff = {}

In [84]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(5)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
list_text = soup.find('div', class_='list_text')
news_title = list_text.find('a').text
news_p = soup.find('div', class_='article_teaser_body').text

In [85]:
mars_stuff["news_title"] = news_title
mars_stuff["news_p"] = news_p

# JPL Mars Space Images - Featured Image
* Visit the url for JPL Featured Space Image here https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars.
* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
* Make sure to find the image url to the full size .jpg image.
* Make sure to save a complete url string for this image.

In [86]:
pic_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(pic_url)
pic_html = browser.html
soup = BeautifulSoup(pic_html, 'html.parser')

image = soup.find("a", class_="button fancybox")["data-link"]
step_two = "https://jpl.nasa.gov"+image
browser.visit(step_two)
step_two_html = browser.html
soup = BeautifulSoup(step_two_html, 'html.parser')

lede = soup.find("figure", class_="lede")
pic_title = lede.find('a')['href']
featured_image_url = "https://jpl.nasa.gov"+pic_title

In [87]:
mars_stuff["featured_image_url"] = featured_image_url

# Mars Weather
* Visit the Mars Weather twitter account here (https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.

* Note: Be sure you are not signed in to twitter, or scraping may become more difficult.

* Note: Twitter frequently changes how information is presented on their website. If you are having difficulty getting the correct html tag data, consider researching Regular Expression Patterns and how they can be used in combination with the .find() method.

In [88]:
twit_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twit_url)
while True:
    if not browser.is_element_not_present_by_tag('article'):
        break
twit_html = browser.html
soup = BeautifulSoup(twit_html, 'html.parser')
tweets = soup.find('article')
for tweet in tweets:
    spans = tweet.find_all("span")
    mars_weather = spans[4].get_text()

In [89]:
mars_stuff["mars_weather"] = mars_weather

# Mars Facts

* Visit the Mars Facts webpage here (https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
* Use Pandas to convert the data to a HTML table string.

In [90]:
fact_url = 'https://space-facts.com/mars/'
browser.visit(fact_url)
fact_html = browser.html
soup = BeautifulSoup(fact_html, 'html.parser')
# print(soup.prettify())
facts=pd.read_html(fact_url)
mars_data=pd.DataFrame(facts[0])
mars_data.columns=['description','value']
mars_table=mars_data.set_index("description")
marsdata = mars_table.to_html(classes='marsdata')
marsdata=marsdata.replace('\n', '')

In [91]:
mars_stuff["mars_facts"] = marsdata

# Mars Hemispheres
* Visit the USGS Astrogeology site here (https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.
* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [92]:
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)
hemi_html = browser.html
soup = BeautifulSoup(hemi_html, 'html.parser')

# print(soup.prettify())
hemisphere_image_urls=[]
for thumb in range (4):
    #time.sleep(5)
    images = browser.find_by_tag('h3')
    images[thumb].click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    full_guy = soup.find("li")
    img_url = full_guy.find('a')['href']
    img_title = soup.find("h2",class_="title").text
    hemisphere={
        "title":img_title,
        "img_url":img_url
    }
    hemisphere_image_urls.append(hemisphere)
    browser.back()

In [93]:
mars_stuff['hemisphere_image_urls'] = hemisphere_image_urls

In [94]:
mars_stuff

{'news_title': "NASA's Mars Rover Drivers Need Your Help",
 'news_p': 'Using an online tool to label Martian terrain types, you can train an artificial intelligence algorithm that could improve the way engineers guide the Curiosity rover.',
 'featured_image_url': 'https://jpl.nasa.gov/spaceimages/images/largesize/PIA03519_hires.jpg',
 'mars_weather': 'How about a little pick-me-up courtesy of these talented young people.',
 'mars_facts': '<table border="1" class="dataframe marsdata">  <thead>    <tr style="text-align: right;">      <th></th>      <th>value</th>    </tr>    <tr>      <th>description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <